In [ ]:
import os

In [ ]:
dataset_id: str = "dummy-ecommerce"
taxonomy_labels = ["electronics", "clothes", "beauty"]
fields = ["product_title"]
output_field: str = "label"
authorizationToken: str = f"{os.environ['TEST_ACTIVATION_TOKEN']}" if os.getenv('TEST_ACTIVATION_TOKEN') else ""

max_number_of_labels: int = 1
threshold: float = 0.5
token: str = ""


In [ ]:
from relevanceai.utils import decode_workflow_token
config = decode_workflow_token(token)
model_id: str = config.get("model_id", "princeton-nlp/sup-simcse-roberta-large")
fields = config.get('fields')
output_field = config.get("output_field")

if output_field == "":
    output_field = None

from relevanceai import Client
client = Client(token=config['authorizationToken'])
ds = client.Dataset(config['dataset_id'])
from sentence_transformers import SentenceTransformer
enc = SentenceTransformer(model_id)
label_documents = [{"label": l, "label_vector_": enc.encode(l).tolist()} for l in taxonomy_labels]
for field in fields:
    if not field.endswith("_vector_"):
        ds.vectorize_text(fields=[field], models=[model_id])

    if not field.endswith("_vector_"):
        # rename to a different vector 
        field = field + "_" + model_id + "_vector_"
        
    vector_field = field.replace("/", "_")
    ds.label(
      vector_fields=[vector_field],
      label_documents=label_documents,
      max_number_of_labels=max_number_of_labels,
      output_field=output_field,
      similarity_threshold=threshold
    )